## The _Kata_ of the Golden Rose

This _kata_ is a classic _refactoring_ exercise: we have a program that works, but it is necessary to add a new use case. For reasons that are self-explanatory by reading the program, part of it is required to be rewritten.

Originally conceived by [@TerryHughes](https://twitter.com/TerryHughes) and [@NotMyself](https://twitter.com/NotMyself) as an exercise in [_refactoring_ and C#](https://github.com/NotMyself/GildedRoseCore), [was extended to different languages](https://github.com/emilybache/GildedRose-Refactoring-Kata.git) and included in the book [Coding Dojo](https://leanpub.com/codingdojohandbook) by Emily Bache.

In this notebook you will find
- The original description of the code specifications, requirements and the new use case.
- The code itself to explore.
- Some comments about object oriented programming and mutability from an F# point of view.

the mission is
- Write tests to prove that the code is actually working for the use cases for which it was designed.
- Write new tests to address the new case ('conjured' articles).
- Do the refactoring.

for this, [this repository](https://github.com/fcolavecchia/GildedRose-Kata-initial-fsharp.git) is provided, which contains three projects
- `GildedRose` with the original program.
- `GildedRose.UnitTests` where you can write the unit tests.
- `GildedRose.ApprovalTests` which contains the code approval test (for the original use cases.)

Fork accordingly and happy refactoring!





### Specifications of the Golden Rose (Gilded Rose)

Welcome to the **Gilded Rose** team.
As you may know, we are a small inn strategically located in a prestigious city, run by the friendly **Allison**.
We also buy and sell high quality merchandise.
Unfortunately, our merchandise is declining in quality as the sale date approaches.

We have a system installed that automatically updates the `inventory`.
This system was developed by a boy with little common sense named Leeroy, who is now off on new adventures.
Your task is to add a new feature to the system so that we can start selling a new category of items.

### Preliminary Description

But first, let's introduce the system:

* All items (`Item`) have a `sellIn` property that denotes the number of days we have to sell it
* All items have a `quality` property that denotes how valuable the item is
* At the end of each day, our system decrements both values ​​for each item using the `updateQuality` method

Pretty simple, right? Well, now is where it gets interesting:

* Once the recommended sell-by date has passed, `quality` degrades at twice the rate
* The `quality` of an article is never negative
* "Aged Brie Cheese" (`Aged brie`) increases its `quality` as it gets older
* Its `quality` increases by `1` unit every day
* after the `sale date` its `quality` increases by `2` units per day
* The `quality` of an article is never greater than `50`
* The item "Sulfuras" (`Sulfuras`), being a legendary item, does not change its `sales date` or degrade in `quality`
* A "Backstage Ticket", like brie, increases in `quality` as the `sell by` date approaches
  * if the concert is 10 days or less away, the `quality` is increased by `2` units
  * if there are 5 days or less left, the `quality` is increased by `3` units
  * after the `sale date` the `quality` drops to `0`

### The requirement

We recently hired a vendor of *magically conjured* items.
This requires a system update:

* `Sorcered` items degrade their `quality` at twice the rate of normal items

Feel free to make any changes to the `updateQuality` message and add code as necessary, as long as everything is working correctly. However, **do not alter the `Item` object or its properties** as they belong to the goblin in that corner, who in a fit of rage will smack you down because he does not believe in codeshare culture .

### Endnotes

To clarify: an item can never have a `quality` higher than `50`, however Sulfuras being a legendary item has an immutable quality of `80`.

## Classes and Mutability

Below you can find the key parts of the code of this _kata_ that need to be refactored. A glimpse to it will reveal some features of the language (and the .NET ecosystem) that need to be introduced. This is due to the fact that this F# code is a _mutatis mutandis_ translation of the original code written in C#. Let us inspect the code and check the new stuff. Bear in mind that F# is a _flexible_ language, and has some features that can enable one to write mutable code and object oriented classes. Although this drifts apart to some extent from a more pure functional paradigm, it is important at this point to recognize that one can deal with mutability in F#, the key is to take control of mutability where it is convenient, and proceed with functional style, when it is convenient too. 

Anyway, in this exercise, we aim to transform the code to a full functional inmutable version.

In [21]:
open System.Collections.Generic

type Item = 
    {
        Name: string
        SellIn: int 
        Quality: int 
    }

First thing first, the type `Item` is a typical record, no big deal here. But, we are opening [`System.Collections.Generic`](https://learn.microsoft.com/en-us/dotnet/api/system.collections.generic?view=net-7.0), that organizes the generic type from which all the collection types in .NET are derived.

Next, a list of items is defined. Note that this is not our usual F# list, but a `System.Collection.Generic.List` which is mutable, and provides a method `.Add` that appends a new item into the list, in a mutable way. Note also the use of the keyword `new` to create (invocate the constructor) of the empty `List<Item>`. 

In [22]:
let Items = new List<Item>()
Items.Add({Name = "+5 Dexterity Vest"; SellIn = 10; Quality = 20})
Items.Add({Name = "Aged Brie"; SellIn = 2; Quality = 0})
Items.Add({Name = "Elixir of the Mongoose"; SellIn = 5; Quality = 7})
Items.Add({Name = "Sulfuras, Hand of Ragnaros"; SellIn = 0; Quality = 80})
Items.Add({Name = "Sulfuras, Hand of Ragnaros"; SellIn = -1; Quality = 80})
Items.Add({Name = "Backstage passes to a TAFKAL80ETC concert"; SellIn = 15; Quality = 20})
Items.Add({Name = "Backstage passes to a TAFKAL80ETC concert"; SellIn = 10; Quality = 49})
Items.Add({Name = "Backstage passes to a TAFKAL80ETC concert"; SellIn = 5; Quality = 49})
Items.Add({Name = "Conjured Mana Cake"; SellIn = 3; Quality = 6})

printfn "%A" Items


seq
  [{ Name = "+5 Dexterity Vest"
     SellIn = 10
     Quality = 20 }; { Name = "Aged Brie"
                       SellIn = 2
                       Quality = 0 }; { Name = "Elixir of the Mongoose"
                                        SellIn = 5
                                        Quality = 7 };
   { Name = "Sulfuras, Hand of Ragnaros"
     SellIn = 0
     Quality = 80 }; ...]


To avoid confusion between these two different types of list, the `ResizeArray` type is used in F#, that is exactly the same as before:

In [23]:
let primes = ResizeArray<int>()
primes.Add(2)
primes.Add(3)
primes.Add(5)
// etc. 
printfn "%A" primes

seq [2; 3; 5]


The `ResizeArray` is mutable, the specific symbol `<-` is used to change a value -in place_:

In [24]:
primes[0] <- 7

printfn "%A" primes

seq [7; 3; 5]


One can use the keyword `mutable` to specify that a value is mutable, obviously:

In [25]:
let mutable a = 3 
printfn "original a: %A" a
a <- 4
printfn "mutated  a: %A" a

original a: 3
mutated  a: 4


Back to the Gilded Rose. Here it is the type `GildedRose`:

In [26]:
type GildedRose(items:IList<Item>) =
    let Items = items

    member this.UpdateQuality() =
        for i = 0 to Items.Count - 1 do
            if Items.[i].Name <> "Aged Brie" && Items.[i].Name <> "Backstage passes to a TAFKAL80ETC concert" then
                if Items.[i].Quality > 0 then
                    if Items.[i].Name <> "Sulfuras, Hand of Ragnaros" then
                        Items.[i] <- { Items.[i] with Quality = (Items.[i].Quality - 1) } 
            else
               if Items.[i].Quality < 50 then
                    Items.[i] <- { Items.[i] with Quality = (Items.[i].Quality + 1) } 
                    if Items.[i].Name = "Backstage passes to a TAFKAL80ETC concert" then
                        if Items.[i].SellIn < 11 then
                            if Items.[i].Quality < 50 then
                                Items.[i] <- { Items.[i] with Quality = (Items.[i].Quality + 1) } 
                        if Items.[i].SellIn < 6 then
                            if Items.[i].Quality < 50 then
                                Items.[i] <- { Items.[i] with Quality = (Items.[i].Quality + 1) } 
            if Items.[i].Name <> "Sulfuras, Hand of Ragnaros" then                 
                Items.[i] <- { Items.[i] with SellIn  = (Items.[i].SellIn - 1) } 
            if Items.[i].SellIn < 0 then
                if Items.[i].Name <> "Aged Brie" then
                    if Items.[i].Name <> "Backstage passes to a TAFKAL80ETC concert" then
                        if Items.[i].Quality > 0 then
                            if Items.[i].Name <> "Sulfuras, Hand of Ragnaros" then
                                Items.[i] <- { Items.[i] with Quality   = (Items.[i].Quality  - 1) } 
                    else
                        Items.[i] <- { Items.[i] with Quality   = (Items.[i].Quality  - Items.[i].Quality) } 
                else
                    if Items.[i].Quality < 50 then
                        Items.[i] <- { Items.[i] with Quality   = (Items.[i].Quality + 1) }  
        ()


Few things to notice here

- The type `GildedRose` takes a value `item` which is an `Item` ResizeArray as a parameter to its constructor.
  
- The code takes advantage of the mutability of the `Items` ResizeArray (i.e., the `System.Collections.Generic.List`) heavily. Not to clear to read though.
- The type `GildedRose` has a [_member_](https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/members/methods) function, pretty much the same as in a typicall object-oriented class. The method `UpdateQuality` is an instance method, and can be called as:

```fsharp
let gildedRose = new GildedRose(Items)
gildedRose.UpdateQuality()
```

One can also have _static member_, for example:

In [27]:
type Point(x,y) = 
    member this.X = x
    member this.Y = y
    static member Length (p:Point) = sqrt(float(p.X*p.X + p.Y*p.Y))

let a = Point(1.0,2.0)
printfn "the x coordinate of a is %A" a.X 
printfn "the y coordinate of a is %A" a.Y 
printfn "the length of a is %A" (Point.Length a) 


the x coordinate of a is 1.0
the y coordinate of a is 2.0
the length of a is 2.236067977


We can see also the use of the `for ..to .. do` construct, which is the usual loop construct. This is also used in the main code of the exercise:

In [28]:
let app = new GildedRose(Items)
for i = 0 to 30 do
    printfn "-------- day %d --------" i
    printfn "name, sellIn, quality"
    for j = 0 to Items.Count - 1 do
        printfn "%s, %d, %d" Items.[j].Name Items.[j].SellIn Items.[j].Quality
    printfn ""
    app.UpdateQuality()

-------- day 0 --------
name, sellIn, quality
+5 Dexterity Vest, 10, 20
Aged Brie, 2, 0
Elixir of the Mongoose, 5, 7
Sulfuras, Hand of Ragnaros, 0, 80
Sulfuras, Hand of Ragnaros, -1, 80
Backstage passes to a TAFKAL80ETC concert, 15, 20
Backstage passes to a TAFKAL80ETC concert, 10, 49
Backstage passes to a TAFKAL80ETC concert, 5, 49
Conjured Mana Cake, 3, 6

-------- day 1 --------
name, sellIn, quality
+5 Dexterity Vest, 9, 19
Aged Brie, 1, 1
Elixir of the Mongoose, 4, 6
Sulfuras, Hand of Ragnaros, 0, 80
Sulfuras, Hand of Ragnaros, -1, 80
Backstage passes to a TAFKAL80ETC concert, 14, 21
Backstage passes to a TAFKAL80ETC concert, 9, 50
Backstage passes to a TAFKAL80ETC concert, 4, 50
Conjured Mana Cake, 2, 5

-------- day 2 --------
name, sellIn, quality
+5 Dexterity Vest, 8, 18
Aged Brie, 0, 2
Elixir of the Mongoose, 3, 5
Sulfuras, Hand of Ragnaros, 0, 80
Sulfuras, Hand of Ragnaros, -1, 80
Backstage passes to a TAFKAL80ETC concert, 13, 22
Backstage passes to a TAFKAL80ETC concert, 8,

## When to Use Classes, Unions, Records, and Structures

For completeness, I transcribe [the following paragraphs from the documentation](https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/classes#when-to-use-classes-unions-records-and-structures), which hopefully clarify the use of types and classes:

Given the variety of types to choose from, you need to have a good understanding of what each type is designed for to select the appropriate type for a particular situation. Classes are designed for use in object-oriented programming contexts. Object-oriented programming is the dominant paradigm used in applications that are written for the .NET Framework. If your F# code has to work closely with the .NET Framework or another object-oriented library, and especially if you have to extend from an object-oriented type system such as a UI library, classes are probably appropriate.

If you are not interoperating closely with object-oriented code, or if you are writing code that is self-contained and therefore protected from frequent interaction with object-oriented code, you should consider using a mix of classes, records and discriminated unions. A single, well thought–out discriminated union, together with appropriate pattern matching code, can often be used as a simpler alternative to an object hierarchy. For more information about discriminated unions, see Discriminated Unions.

Records have the advantage of being simpler than classes, but records are not appropriate when the demands of a type exceed what can be accomplished with their simplicity. Records are basically simple aggregates of values, without separate constructors that can perform custom actions, without hidden fields, and without inheritance or interface implementations. Although members such as properties and methods can be added to records to make their behavior more complex, the fields stored in a record are still a simple aggregate of values. For more information about records, see Records.

Structures are also useful for small aggregates of data, but they differ from classes and records in that they are .NET value types. Classes and records are .NET reference types. The semantics of value types and reference types are different in that value types are passed by value. This means that they are copied bit for bit when they are passed as a parameter or returned from a function. They are also stored on the stack or, if they are used as a field, embedded inside the parent object instead of stored in their own separate location on the heap. Therefore, structures are appropriate for frequently accessed data when the overhead of accessing the heap is a problem. For more information about structures, see Structs.

